# 💡 
If outlier exposure is used does a neural network perform better? The hypothesis is that it will learn a more general representation that won't be interfered with as much.

In [93]:
import typing
import torch
import torchvision.transforms as transforms 
import torchvision.datasets
from random import shuffle
import torch.nn as nn
from torch import Tensor
from dataclasses import dataclass
from avalanche.benchmarks.classic.cfashion_mnist import SplitFMNIST
from avalanche.training.templates.supervised import SupervisedTemplate
from avalanche.benchmarks.utils import AvalancheSubset, AvalancheDataset
from avalanche.benchmarks.scenarios.new_classes.nc_scenario import NCScenario
from avalanche.training.plugins.lwf import LwFPlugin


%load_ext autoreload
%autoreload 2

from conf import *
from experiment.experiment import Experiment, BaseHyperParameters
from functional.functional import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:
class MySimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()

        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(28*28, 512),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(256, 10),
            nn.Dropout(0.5),
            nn.ReLU(),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.network(x)

In [110]:
@dataclass
class HyperParams(BaseHyperParameters):
    exposure_set_size: int
    weight_decay: float

class OE_Experiment(Experiment):

    hp: HyperParams
    network: MySimpleMLP

    classifier_weight: float = 50.0

    def __init__(self, hp: HyperParams) -> None:
        super().__init__(hp)

        self.after_eval_forward = self.after_forward

    def make_network(self) -> nn.Module:
        return MySimpleMLP()

    def make_optimizer(self, parameters) -> torch.optim.Optimizer:
        optimizer = torch.optim.SGD(parameters, self.hp.lr, weight_decay=self.hp.weight_decay)
        return optimizer

    def make_criterion(self):
        return softTargetCrossEntropy

    def make_scenario(self):
        # transform = transforms.Compose(
        #     [transforms.ToTensor(), transforms.Normalize((0.2860,), (0.3530,)), transforms.Resize(32)]
        # )
        scenario: NCScenario = SplitFMNIST(
            n_experiences=5,
            fixed_class_order=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
            dataset_root=DATASETS)

        # Use a uniform distribution for outlier exposure
        return scenario

    def add_plugins(self):
        return [LwFPlugin()]
        

    def known_known(self, n_classes: int):
        def _known_known(y: int) -> Tensor:
            new_y = torch.zeros(n_classes)
            new_y[y] = 1.0
            return new_y
        return _known_known

    def known_unkown(self, n_classes: int, not_ys: typing.Sequence[int]):
        def known_unkown(y: int) -> Tensor:
            # return y
            y = torch.ones(n_classes) * 1/(n_classes-len(not_ys))
            for not_y in not_ys:
                y[not_y] = 0.0
            return y
        return known_unkown

    def after_train_dataset_adaptation(self, strategy: SupervisedTemplate):
        scenario: NCScenario = strategy.experience.benchmark
        current_dataset: AvalancheSubset = strategy.adapted_dataset
        n_classes: int = scenario.n_classes

        # Build set of outlier exposures
        exposure_set: AvalancheSubset  = None
        for i, dataset in enumerate(
            map(lambda x: x.dataset, scenario.train_stream)):
            dataset: AvalancheSubset

            # Don't use already seen instances for outlier exposure
            if i <= strategy.clock.train_exp_counter:
                continue

            dataset = dataset.add_transforms(
                target_transform=self.known_unkown(
                    n_classes,
                    strategy.experience.classes_in_this_experience))
            exposure_set = exposure_set + dataset \
                if exposure_set is not None else dataset 

            print(f"Using experience {i} for outlier exposure")

        current_dataset = current_dataset.add_transforms(target_transform=self.known_known(n_classes))

        # If no exposure set is generated
        if exposure_set is None:
            strategy.adapted_dataset = AvalancheDataset(current_dataset)
            return

        # Random indices ordering 
        indices = list(range(len(exposure_set)))
        shuffle(indices)
        indices = indices[:self.hp.exposure_set_size]
        exposure_set = AvalancheSubset(exposure_set, indices)
        strategy.adapted_dataset = exposure_set + current_dataset

    def after_eval_dataset_adaptation(self, strategy):
        n_classes =  strategy.experience.benchmark.n_classes
        strategy.adapted_dataset = strategy.adapted_dataset.add_transforms(
            target_transform=self.known_known(n_classes))

        # self.after_eval




In [108]:

experiment = OE_Experiment(
    HyperParams(
        lr=0.01,
        train_mb_size=128,
        train_epochs=10,
        eval_mb_size=128,
        eval_every=-1,
        device="cuda",
        exposure_set_size=12000,
        weight_decay=0.0
    )
).train()

/Scratch/al183/avalanche/avalanche/training/templates/base.py:200: UserWarning: Plugin <__main__.OE_Experiment object at 0x7f70aea49c10> implements incompatible callbacks for template <avalanche.training.templates.supervised.SupervisedTemplate object at 0x7f70ad76bac0>. This may result in errors.
  warnings.warn(


Start of experience: 0
Current Classes:     [0, 1]
Experience size:     12000
Using experience 1 for outlier exposure
Using experience 2 for outlier exposure
Using experience 3 for outlier exposure
Using experience 4 for outlier exposure
Start of experience: 1
Current Classes:     [2, 3]
Experience size:     12000
Using experience 2 for outlier exposure
Using experience 3 for outlier exposure
Using experience 4 for outlier exposure
Start of experience: 2
Current Classes:     [4, 5]
Experience size:     12000
Using experience 3 for outlier exposure
Using experience 4 for outlier exposure
Start of experience: 3
Current Classes:     [6, 7]
Experience size:     12000
Using experience 4 for outlier exposure


KeyboardInterrupt: 